In [ ]:
import os
from dotenv import load_dotenv
from typing import Any

from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIResponsesClient

from agent_framework import Message, WorkflowEvent
from agent_framework.orchestrations import ConcurrentBuilder

In [ ]:
load_dotenv(override=True)

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

In [ ]:
credential = AzureCliCredential()
chat_client = AzureOpenAIResponsesClient(
    project_endpoint=project_endpoint,
    deployment_name=model,
    credential=credential,
)

In [ ]:
researcher = chat_client.as_agent(
    instructions=(
        "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
        " opportunities, and risks."
    ),
    name="researcher",
)

marketer = chat_client.as_agent(
    instructions=(
        "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
        " aligned to the prompt."
    ),
    name="marketer",
)

legal = chat_client.as_agent(
    instructions=(
        "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
        " based on the prompt."
    ),
    name="legal",
)

In [ ]:
# Define a custom aggregator callback that uses the chat client to summarize
async def summarize_results(results: list[Any]) -> str:
    # Extract one final assistant message per agent
    expert_sections: list[str] = []
    for r in results:
        try:
            messages = getattr(r.agent_run_response, "messages", [])
            final_text = messages[-1].text if messages and hasattr(messages[-1], "text") else "(no content)"
            expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}:\n{final_text}")
        except Exception as e:
            expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}: (error: {type(e).__name__}: {e})")

    # Ask the model to synthesize a concise summary of the experts' outputs
    system_msg = Message(
        role="system",
        contents=[
            """
            You are a helpful assistant that consolidates multiple domain expert outputs into one cohesive, 
            concise summary with clear takeaways. Keep it under 200 words.
            """],
    )
    user_msg = Message(role="user", contents=["\n\n".join(expert_sections)])

    response = await chat_client.get_response([system_msg, user_msg])
    # Return the model's final assistant text as the completion result
    return response.messages[-1].text if response.messages else ""

In [ ]:
workflow = (
    ConcurrentBuilder(participants=[researcher, marketer, legal])
    .with_aggregator(summarize_results)
    .build()
)

In [ ]:
output_evt: WorkflowEvent | None = None
query = "We are launching a new budget-friendly electric bike for urban and rural commuters."

async for event in workflow.run(query, stream=True):
    if event.type == "output":
        output_evt = event

In [ ]:
if output_evt:
    print("===== Final Consolidated Output =====")
    print(output_evt.data)